In [1]:
# import
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
#print(plt.style.available)
plt.style.use(plt.style.available[7])



# scalar var, init
# number of individu
n_inv = 200
# infection ratio, in the start of simulation
inf_rto = 0.05
# recovery time for an infected ind
recov_tm =10
# x, y range
x_range = [0,20]
y_range = [0,20]
# moving probability
moving_prob = 0.8


# arr var , init
# list of individu
ind_list = []
# list of health stat
health_stat = []
# list imunity stat of an individu
imunity_stat = []
# list of count down to recovery 
inf_ctd = []


# aditional data
# inf, healty list
line_plot_data = [n_inv * inf_rto]

# iter list
iter_list = [int(0)]

In [2]:
# init random starting point
for i in range(n_inv) :
    # starting point is with in the simulation boundry 
    ind_list.append([[np.random.randint(0,20), np.random.randint(0,20)]])

# init healt_stat , inf_ctd & imunity_stat
for i in range(n_inv) :
    if sum(health_stat) < (n_inv * inf_rto) :
        if bool(np.random.randint(0,2)) :
            health_stat.append(1)
        else :
            health_stat.append(0)
    else :
        health_stat.append(0)
    imunity_stat.append(0)
    inf_ctd.append(0)
        

In [3]:
# colour code for infected and not infected
colour_code = ['blue', 'red']


# main loop
while (sum(health_stat)) > 0 :
    # draw first
    
    # plt figure init 
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8), dpi=72)
    
    # plot healty individu
    # iterate through individu coordinate -> every point
    for idx in range(len(ind_list)) :
        # scatter the latest point of healty individu
        if health_stat[idx] == 0 :
            axes[0].scatter(x=ind_list[idx][-1][0] , y=ind_list[idx][-1][1] , color=colour_code[health_stat[idx]])
    
    # why is it double ?
    # so the infected ind more visible -> not overlapped with healty individu
    # plot infected individu
    # iterate through individu coordinate -> every point
    for idx in range(len(ind_list)) :
        # scatter the latest point of healty individu
        if health_stat[idx] == 1 :
            axes[0].scatter(x=ind_list[idx][-1][0] , y=ind_list[idx][-1][1] , color=colour_code[health_stat[idx]])
    # plot infected rate
    axes[1].plot(iter_list,line_plot_data,color='red')
    
    # subplot 0 limit
    axes[0].set_xlim([x_range[0]-1, x_range[1]+1])
    axes[0].set_ylim([y_range[0]-1, y_range[1]+1])

    # Set axis labels
    axes[0].set_xlabel('X', fontsize=18)
    axes[0].set_ylabel('Y', fontsize=18)

    axes[1].set_xlabel('Days', fontsize=18)
    axes[1].set_ylabel('Infected', fontsize=18)

    # set subplot title
    axes[0].title.set_text('Walk iter %d' % iter_list[-1])
    axes[1].title.set_text('Infected Plot iter %d' % iter_list[-1])
    
    # fontsize of minor  and major ticks label
    # of all subplot
    axes[0].tick_params(axis='both', which='major', labelsize=14)
    axes[0].tick_params(axis='both', which='minor', labelsize=14)
    
    axes[1].tick_params(axis='both', which='major', labelsize=14)
    axes[1].tick_params(axis='both', which='minor', labelsize=14)
    
    # save fig to file 
    plt.savefig('iter_%d.png' % iter_list[-1])
    
    # closing the figure
    plt.close(fig)
    
    
    # advance iter 
    iter_list.append(int(iter_list[-1]+1))
    # moving part
    # iterating through individu in ind_list
    for ind_idx in range(0,n_inv) :
        # starting point
        starting_point = ind_list[ind_idx][-1]
        
        # determine moves up down left right
        # random float
        random_number = np.random.rand()
        
        # copy starting point to another variable
        endPoint = starting_point[:]
        
        # chance to moves
        # check if an individu have to move
        if (np.random.rand() <= moving_prob) :
            # up
            if 0 <= random_number and random_number <= 0.25 :
                endPoint[1] = endPoint[1] +1
            # down
            elif 0.26 <= random_number and random_number <= 0.50 :
                endPoint[1] = endPoint[1] -1
            # left
            elif 0.51 <= random_number and random_number <= 0.75 :
                endPoint[0] = endPoint[0] -1
            # right
            elif 0.75 <= random_number and random_number <= 1 :
                endPoint[0] = endPoint[0] +1
        
        # pbc
        # boundry check
        if endPoint[0] > x_range[1] :
            endPoint[0] = endPoint[0] -1
        elif endPoint[0] < x_range[0] :
            endPoint[0] = endPoint[0] + 1
            
        if endPoint[1] > y_range[1] :
            endPoint[1] = endPoint[1] -1
        elif endPoint[1] < y_range[0] :
            endPoint[1] = endPoint[1] + 1
            
        # append to end of object
        # wheter it move or not still append the data
        ind_list[ind_idx].append(endPoint)
        
        
    
    # update ind status
    # first heal infected
    
    # iterating through inf_ctd list
    for idx in range(len(inf_ctd)) :
        
        # check wheter the individu is sick and have infected for 10 day
        if inf_ctd[idx] >= 10 and health_stat[idx] == 1:
            # set inf_ctd to not infected [0
            inf_ctd[idx] = 0
            # set health_stat to healty [0]
            health_stat[idx] = 0
            # set imunity_stat to 1
            # cant get infected again
            imunity_stat[idx] = 1
        # if the individu is sick
        # and infected for less than 10 days
        elif health_stat[idx] == 1:
            # add 1 to count down
            inf_ctd[idx] = inf_ctd[idx] + 1

    # infect other ind
    # iterate through all individu using index
    for idx in range(len(health_stat)) :
        # check wheter the individu is sick
        if health_stat[idx] == 1 :
            # copy position data of infected individu
            infected_point = ind_list[idx][-1][:]
            
            # chack for overlap in position data to every individu position 
            # with index above idx to the end list
            for idk in range(idx, len(ind_list)) :
                # check if the point overlap, infected_point -> ind_list
                if infected_point == ind_list[idk][-1][:] :
                    # chack wheter the individu that overlap have imunity
                    # if have no imunity
                    if imunity_stat[idk] == 0 :
                        # individu get sick
                        health_stat[idk] = 1
    
    # save data for line plot
    line_plot_data.append(sum(health_stat))

    
# out of the main loop
# draw last day in simulation  
# plt figure init 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8), dpi=72)
    
# plot healty individu
# iterate through individu coordinate -> every point
for idx in range(len(ind_list)) :
    # scatter the latest point of healty individu
    if health_stat[idx] == 0 :
        axes[0].scatter(x=ind_list[idx][-1][0] , y=ind_list[idx][-1][1] , color=colour_code[health_stat[idx]])

# why is it double ?
# so the infected ind more visible -> not overlapped with healty individu
# plot infected individu
# iterate through individu coordinate -> every point
for idx in range(len(ind_list)) :
    # scatter the latest point of healty individu
    if health_stat[idx] == 1 :
        axes[0].scatter(x=ind_list[idx][-1][0] , y=ind_list[idx][-1][1] , color=colour_code[health_stat[idx]])

# plot infected rate
axes[1].plot(iter_list,line_plot_data,color='red')

# subplot 0 limit
axes[0].set_xlim([x_range[0]-1, x_range[1]+1])
axes[0].set_ylim([y_range[0]-1, y_range[1]+1])


# Set labels for subplot
axes[0].set_xlabel('X', fontsize=18)
axes[0].set_ylabel('Y', fontsize=18)
axes[1].set_xlabel('Days', fontsize=18)
axes[1].set_ylabel('Infected', fontsize=18)

# subplot title 
axes[0].title.set_text('Walk iter %d [Last Day]' % (iter_list[-1]))
axes[1].title.set_text('Infected Plot iter %d [Last Day]' % (iter_list[-1]))

# fontsize of minor  and major ticks label
# of all subplot
axes[0].tick_params(axis='both', which='major', labelsize=14)
axes[0].tick_params(axis='both', which='minor', labelsize=14)
axes[1].tick_params(axis='both', which='major', labelsize=14)
axes[1].tick_params(axis='both', which='minor', labelsize=14)

# save figure
plt.savefig('iter_%d.png' % iter_list[-1])

# closing the figure
plt.close(fig)


In [4]:
# animating
# 1.get list of file name
# 2.animate
# 3.remove temp

# list that will contain path to every image
path_list = []

# iterate to every element if iter_list
for i in iter_list :
    # add path found to the list
    path_list.append(os.getcwd()+'\\iter_%d.png' % i)

# add padding of last frame so its not just disapear to ether ~10 %
for i in range(int(0.1*len(path_list) + 1)) :
    path_list.append(path_list[-1])
### part to animate using cv2
# https://stackoverflow.com/questions/44947505/how-to-make-a-movie-out-of-images-in-python
# video name
video_name = 'random_walk_inf.mp4'

# frame -> np array , maybe ?
# check the first image in path_list
frame = cv2.imread(path_list[0])
# get height and width of frame
height, width, layers = frame.shape

# videowriter object to write video
# videoname = video_name
# codec = 0 # none
# fps = 4
# tuple of resolution
video = cv2.VideoWriter(video_name, 0,  4, (width,height))


# iterating through every image path 
# in path_list 
for image in path_list:
    # write every frame in video
    video.write(cv2.imread(image))

    
# ????
# close cv2 windows
cv2.destroyAllWindows()
# release the video
video.release()
    
    
    
    
#### Delete temp file
#x = input('Press Enter to delete temp file ...')
# iterate every path from path_list
for pat in path_list :
    # check is it a file or exist
    if os.path.isfile(pat) :
        # delete the file
        os.remove(pat)       

In [5]:
%%HTML
<!--Magic Cell--!>
<!--https://stackoverflow.com/questions/18019477/how-can-i-play-a-local-video-in-my-ipython-notebook--!>
<video width="996" height="560" controls>
  <source src='random_walk_inf.mp4' type="video/mp4">
</video>